In [1]:
import torchrl
import torch
import gymnasium
from torchrl.modules import ProbabilisticActor, TanhNormal, ValueOperator
from tensordict.nn import TensorDictModule
from tensordict.nn.distributions import NormalParamExtractor
import numpy as np


In [2]:
#env = torchrl.envs.GymEnv("InvertedPendulum-v4",)

In [3]:
	
env = gymnasium.make("InvertedPendulum-v4")

display(env.reward_range,
env.observation_space,
env.action_space)
env.reset()

(-inf, inf)

Box(-inf, inf, (4,), float64)

Box(-3.0, 3.0, (1,), float32)

(array([ 0.00613129, -0.0016405 ,  0.00891457, -0.00967371]), {})

In [15]:
import torchrl.envs


env = env.unwrapped
env= torchrl.envs.GymWrapper(env)
#env = torchrl.envs.GymEnv("InvertedDoublePendulum-v4")

env.reset()

TypeError: Can't instantiate abstract class GymLikeEnv with abstract methods _build_env, _check_kwargs, _init_env, _make_specs, _output_transform, _reset_output_transform, _set_seed

In [5]:
env.action_space.dtype
a = env.action_space.sample()
env.rollout(1)

TensorDict(
    fields={
        action: Tensor(shape=torch.Size([1, 1]), device=cpu, dtype=torch.float32, is_shared=False),
        done: Tensor(shape=torch.Size([1, 1]), device=cpu, dtype=torch.bool, is_shared=False),
        next: TensorDict(
            fields={
                done: Tensor(shape=torch.Size([1, 1]), device=cpu, dtype=torch.bool, is_shared=False),
                observation: Tensor(shape=torch.Size([1, 4]), device=cpu, dtype=torch.float64, is_shared=False),
                reward: Tensor(shape=torch.Size([1, 1]), device=cpu, dtype=torch.float32, is_shared=False),
                terminated: Tensor(shape=torch.Size([1, 1]), device=cpu, dtype=torch.bool, is_shared=False),
                truncated: Tensor(shape=torch.Size([1, 1]), device=cpu, dtype=torch.bool, is_shared=False)},
            batch_size=torch.Size([1]),
            device=cpu,
            is_shared=False),
        observation: Tensor(shape=torch.Size([1, 4]), device=cpu, dtype=torch.float64, is_shared=

In [6]:
from torchrl.envs import (Compose, DoubleToFloat, ObservationNorm, StepCounter,
                          TransformedEnv)
env = TransformedEnv(
    env,
    Compose(
        #normalize observations
        ObservationNorm(in_keys=["observation"]),
        DoubleToFloat(),
        StepCounter(),
    ),
)
env.transform[0].init_stats(num_iter=1000, reduce_dim=0, cat_dim=0)
print("normalization constant shape:", env.transform[0].loc.shape)


normalization constant shape: torch.Size([4])


In [7]:
env.observation_space.dtype

dtype('float64')

In [8]:
a = env.rand_action()
env.step(a)

KeyError: 'key "step_count" not found in TensorDict with keys [\'action\']'

In [ ]:
a = env.action_space.sample()
print(a)
state , reward , _,_,_ = env.step(a)
state.shape

[-2.2885299]


AttributeError: 'numpy.ndarray' object has no attribute 'batch_size'

In [12]:
import torchrl.modules


class myclass(torch.nn.Module):
    
    def forward(self,x):
        x[:,1]=(x[:,1]+1)/2
        return x
actor_net = torch.nn.Sequential(    
    torch.nn.Linear(4,2),
    NormalParamExtractor()
)
policy_module = TensorDictModule(
    actor_net, in_keys=["observation"], out_keys=["loc", "scale"]
)
policy_module= ProbabilisticActor(
    policy_module,
    ['loc','scale'],
    distribution_class=TanhNormal,
    distribution_kwargs={
        "min": env.action_space.low,
        "max": env.action_space.high,
    },
    return_log_prob=True,
    
)


value_net = torch.nn.Sequential(
    torch.nn.Linear(11,1),
    torch.nn.Tanh(),
)
#state = torch.tensor(state.reshape((1,4)),dtype=torch.float32)


value_module = ValueOperator(
    module=value_net,
    in_keys=["observation"],
)


In [13]:
env.reset()['observation']

tensor([ 0.1285,  0.1268,  0.0012, -0.0064])

In [14]:
print("Running policy:", policy_module(env.reset()))
print("Running value:", value_module(env.reset()))

RuntimeError: Can't call numpy() on Tensor that requires grad. Use tensor.detach().numpy() instead.

In [ ]:
from torchrl.collectors.collectors import SyncDataCollector

collator =SyncDataCollector(env,policy_module,frames_per_batch=10,total_frames=100,)

TypeError: unsupported operand type(s) for *: 'numpy.ndarray' and 'Tensor'

In [ ]:
from torchrl.objectives import ClipPPOLoss
from torchrl.objectives.value import GAE

In [ ]:
sub_batch_size = 64  # cardinality of the sub-samples gathered from the current data in the inner loop
num_epochs = 10  # optimization steps per batch of data collected
clip_epsilon = (
    0.2  # clip value for PPO loss: see the equation in the intro for more context.
)
gamma = 0.99
lmbda = 0.95
entropy_eps = 1e-4

advantage_module = GAE(
    gamma=gamma, lmbda=lmbda, value_network=value_module, average_gae=True
)
lr = 1e-5
loss_module = ClipPPOLoss(
    actor_network=policy_module,
    critic_network=value_module,
    clip_epsilon=clip_epsilon,
    entropy_bonus=bool(entropy_eps),
    entropy_coef=entropy_eps,
    # these keys match by default but we set this for completeness
    critic_coef=1.0,
    loss_critic_type="smooth_l1",
)

optim = torch.optim.Adam(loss_module.parameters(), lr)


In [ ]:
collator.next()

KeyError: 'key "action" not found in TensorDict with keys [\'collector\', \'done\', \'loc\', \'observation\', \'scale\', \'terminated\', \'truncated\']'

In [ ]:
import torchrl.trainers


trainer = torchrl.trainers.Trainer(collector=collator,loss_module=loss_module,total_frames=1000,optimizer=optim,frame_skip=0,optim_steps_per_batch=1)
trainer

Trainer(
    loss=ClipPPOLoss(
      (actor_network_params): TensorDictParams(params=TensorDict(
          fields={
              module: TensorDict(
                  fields={
                      0: TensorDict(
                          fields={
                              bias: Parameter(shape=torch.Size([2]), device=cpu, dtype=torch.float64, is_shared=False),
                              weight: Parameter(shape=torch.Size([2, 4]), device=cpu, dtype=torch.float64, is_shared=False)},
                          batch_size=torch.Size([]),
                          device=None,
                          is_shared=False),
                      1: TensorDict(
                          fields={
                          },
                          batch_size=torch.Size([]),
                          device=None,
                          is_shared=False)},
                  batch_size=torch.Size([]),
                  device=None,
                  is_shared=False)},
          batch_si

In [ ]:
trainer.train()

  0%|          | 0/1000 [00:00<?, ?it/s]

KeyError: 'key "action" not found in TensorDict with keys [\'collector\', \'done\', \'loc\', \'observation\', \'scale\', \'terminated\', \'truncated\']'